<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/classify_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run images through image type classification pipeline
---
*Last Updated 30 October 2020*  
Classify images as map, phylogeny, illustration, herbarium sheet, or none.

1) Use "cartoonization" approach to add photographic or non-photographic tags to images. (Low accuracy for illustrations, so this method adds coverage for downstream predictions with low confidence values).

2) Run images through trained MobileNet SSD v2 model to add tags to images for image types (map, phylogeny, ilustration, herbarium sheet, non) for predictions with confidence > 1.6. (Confidence value chosen in [inspect_train_results.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/inspect_train_results.ipynb)).

3) Display tagging results on images to verify behavior is as expected.

### Imports
---

In [ ]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# For working with data and plotting graphs
import itertools
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import norm
from scipy import sum, average
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# For image classification and training
import tensorflow as tf

# For working with images
!pip install pillow
!pip install scipy==1.1.0
import cv2
import scipy
from scipy import misc

#### Define functions & variables
---

In [ ]:
# For images to read in from bundle

# Load in image from URL
# Modified from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb#scrollTo=JhVecdzJTsKE
def image_from_url(url, fn):
  file = tf.keras.utils.get_file(fn, url) # Filename doesn't matter
  disp_img = tf.keras.preprocessing.image.load_img(file)
  img = tf.keras.preprocessing.image.load_img(file, target_size=[224, 224])
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = tf.keras.applications.mobilenet_v2.preprocess_input(
    x[tf.newaxis,...])
  return x, disp_img

# Read in EOL image bundle dataframe
# TO DO: Type in image bundle address using form field to right
bundle = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Angiosperms_20K_breakdown_000031.txt' #@param {type:"string"}
df = pd.read_csv(bundle, sep='\t', header=0)
df.head()

In [ ]:
# For exporting tagging results
import csv

# Write header row of output crops file
# TO DO: Change file name for each bundle/run abcd if doing 4 batches using dropdown form to right
tags_file = "tags_imtype_20k_a" #@param ["tags_imtype_20k_a", "tags_imtype_20k_b", "tags_imtype_20k_c", "tags_imtype_20k_d"]
tags_fpath = "/content/drive/My Drive/summer20/classification/image_type/results/" + tags_file + ".tsv"
with open(tags_fpath, 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  tsv_writer.writerow(["eolMediaURL", "identifier", \
                                       "dataObjectVersionID", "ancestry", \
                                       "mnorm_pp"])

### Cartoonize images to classify as photographic or non-photographic
---   
Cartoonify image, then compare change in color values. If change above a certain threshold, then image is likely a photograph. If change below a certain threshold, image is likely a cartoon (non-photograph).

In [ ]:
from PIL import Image
import time

# TO DO: Set start and end rows to run inference for from EOL image bundle using form field to right
# If running in 4 batches of 5000 images, use values in dropdown menu
a = 0 #@param ["0", "5000", "10000", "15000"] {type:"raw"}
b = 5 #@param ["5", "5000", "10000", "15000", "20000"] {type:"raw"}

# Loop through EOL image bundle to classify images and generate tags
for i, row in df.iloc[a:b].iterrows():
  try:
    # Get url from image bundle
    start = time.time()
    url = df['eolMediaURL'][i]
    # Read in image from url
    fn = str(i) + '.jpg'
    img, disp_img = image_from_url(url, fn)
    # Display image
    #_, ax = plt.subplots(figsize=(10, 10))
    #plt.title("Original")
    #ax.imshow(disp_img)
    # Make edges
    cv_img = np.array(disp_img) 
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5) 
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,  
                                         cv2.THRESH_BINARY, 9, 9)  
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    # Cartoonization - bilateral filter and edges 
    color = cv2.bilateralFilter(cv_img, 9, 250, 250) 
    img2 = cv2.bitwise_and(color, edges) 

    # Calculate difference in original and cartoonized image
    # Convert both images from RGB to HSV
    HSV_img = cv2.cvtColor(cv_img, cv2.COLOR_RGB2HSV)
    HSV_img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2HSV)
    # Fnd the difference for H of HSV values of the images
    diff = HSV_img[:,:,0]-HSV_img2[:,:,0]
    mnorm = sum(abs(diff))  # Manhattan norm
    mnorm_pp = mnorm/HSV_img.size # per pixel
    end = time.time()
    print("Completed for {}, {} of {} files in {} seconds".format(url, i, format(b-a, '.0f'), format(end-start, '.2f')))

    # Display cartoonized image
    #_, ax = plt.subplots(figsize=(10, 10))
    #plt.title("Cartoonized \n Manhattan norm: {} / per pixel {} \
    #".format(mnorm, mnorm_pp))
    #ax.imshow(img2)

    # Export tagging results to tsv
    # Define variables for export
    identifier = df['identifier'][i]
    dataObjectVersionID = df['dataObjectVersionID'][i]
    ancestry = df['ancestry'][i]
    with open(tags_fpath, 'a') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow([url, identifier, dataObjectVersionID, ancestry, \
                               mnorm_pp])

  except:
    print('Check if URL from {} is valid'.format(url))

### Run images through model(s) for image type classification
---   
Use model selected in inspect_train_results.ipynb (MobileNet SSD v2) to classify images as map, phylogeny, herbarium sheet, illustration, or none.

#### Run 20K image bundle through classification pipeline

In [ ]:
# Load trained model from path
TRAIN_SESS_NUM = "13"
saved_model_path = '/content/drive/My Drive/summer20/classification/image_type/saved_models/' + TRAIN_SESS_NUM
imtype_model = tf.keras.models.load_model(saved_model_path)
label_names = ['herbarium sheet', 'illustration', 'map', 'null', 'phylogeny']
module_selection = ("mobilenet_v2_1.0_224", 224)
handle_base, pixels = module_selection
IMAGE_SIZE = (pixels, pixels)

In [ ]:
# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)
import time

# For exporting results
confs = []
imclasses = []

# Loop through EOL image bundle to classify images and generate tags
for i, row in df.iloc[a:b].iterrows():
  try:
    # Get url from image bundle
    url = df['eolMediaURL'][i]
    # Read in image from url
    fn = str(i) + '.jpg'
    img, disp_img = image_from_url(url, fn)
    ax.imshow(disp_img)
    # Record inference time
    start_time = time.time()
    # Detection and draw boxes on image
    # For flowers/fruits (reproductive structures)
    predictions = imtype_model.predict(img, batch_size=1)
    label_num = np.argmax(predictions)
    conf = predictions[0][label_num]
    confs.append(conf)
    imclass = label_names[label_num]
    imclasses.append(imclass)
    end_time = time.time()
    # Display progress message after each image
    print('Inference complete for {} of {} images in {} sec'.format(i, (b-a), \
                                            format(end_time-start_time, '.2f')))

    # Optional: Show classification results for images
    # Only use to view predictions on <50 images at a time
    #_, ax = plt.subplots(figsize=(10, 10))
    #ax.imshow(disp_img)
    #plt.axis('off')
    #plt.title("{}) Prediction: {}, Confidence: {}%, \
    #\n Inference Time: {}".format(i, imclass, conf, \
    #format(end_time-start_time, '.2f')))

  except:
    print('Check if URL from {} is valid'.format(url))

# Export tagging results to tsv
# Define variables for export
classif = pd.DataFrame(([imclasses,confs]))
classif = classif.transpose()
classif.columns = ["imclass", "conf"]
df = pd.read_csv(tags_fpath, sep='\t')
comb = pd.concat([df, classif], axis=1, ignore_index=True)
comb.columns = ["eolMediaURL", "identifier", "dataObjectVersionID", \
                  "ancestry", "mnorm_pp", "imclass", "conf"]
print(comb.head())
comb.to_csv(tags_fpath, sep='\t', index=False)

### Post-process classification predictions using Manhattan norm and confidence threshold values
---
Manhattan norm per pixel threshold (=<2) was determined in cartoonify_images.ipynb. MobileNet SSD v2 confidence threshold (>1.6) was chosen in inspect_train_results.ipynb.

In [ ]:
# Combine exported model predictions and confidence values from above to one dataframe
base = '/content/drive/My Drive/summer20/classification/image_type/results/tags_imtype_20k_' 
exts = ['a.tsv', 'b.tsv', 'c.tsv', 'd.tsv']
all_filenames = [base + e for e in exts]
df = pd.concat([pd.read_csv(f, sep='\t', header=0, na_filter = False) for f in all_filenames], ignore_index=True)

# Filter predictions using determined confidence value thresholds
# Make column for "reproductive structures present?" tag
df['tag_cartoon'] = np.nan
df['tag_imtype'] = np.nan
df['problematic'] = np.nan
# Adjust final tag based on Model 7 and 11 predictions and confidence values
for i, row in df.iterrows():
  # If Manhattan norm per pixel from cartoonization =< 2
  if df['mnorm_pp'][i]<=2: 
    df['tag_cartoon'][i] = 'non-photo'
  elif df['mnorm_pp'][i]>2: 
    df['tag_cartoon'][i] = 'photo'
for i, row in df.iterrows():
  if df['conf'][i]>1.6: 
    df['tag_imtype'][i] = df['imclass'][i]  
  elif df['conf'][i]<=1.6: 
    df['tag_imtype'][i] = 'none' 
for i, row in df.iterrows():
  if df['imclass'][i]=='illustration': 
    df['problematic'][i] = 'maybe' 
for i, row in df.iterrows():
  if 'Chloroplastida' in df['ancestry'][i]: 
    df['problematic'][i] = 'maybe'

# Write results to tsv
df.to_csv("/content/drive/My Drive/summer20/classification/image_type/results/tags_imtype_20k_finaltags.tsv", sep='\t', index=False)

### Display final classification results on images
---

In [ ]:
# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# TO DO: Update file path to finaltags.tsv file
path = "/content/drive/My Drive/summer20/classification/image_type/results/"
f = "tags_imtype_20k_finaltags.tsv" #@param
fpath = path + f
df = pd.read_csv(fpath, sep='\t', header=0, na_filter = False)

# Function to load in image from URL
# Modified from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb#scrollTo=JhVecdzJTsKE
def image_from_url(url, fn):
  file = tf.keras.utils.get_file(fn, url) # Filename doesn't matter
  disp_img = tf.keras.preprocessing.image.load_img(file)
  img = tf.keras.preprocessing.image.load_img(file, target_size=[224, 224])
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = tf.keras.applications.mobilenet_v2.preprocess_input(
    x[tf.newaxis,...])
  return x, disp_img

# TO DO: Set start and end rows to run inference for from EOL image bundle using form field to right
# If running in 4 batches of 5000 images, use values in dropdown menu
start =  0#@param {type:"raw"}
end = 50 #@param {type:"raw"}

# Loop through EOL image bundle to classify images and generate tags
for i, row in df.iloc[start:end].iterrows():
  try:
    # Get url from image bundle
    url = df['eolMediaURL'][i]
    # Read in image from url
    fn = str(i) + '.jpg'
    img, disp_img = image_from_url(url, fn)
    # Record inference time
    tag1 = df['tag_cartoon'][i]
    tag2 = df['tag_imtype'][i]
    # Display progress message after each image is loaded
    print('Successfully loaded {} of {} images'.format(i+1, (end-start)))

    # Show classification results for images
    # Only use to view predictions on <50 images at a time
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(disp_img)
    plt.axis('off')
    plt.title("{}) Photo or not: {}, Image type: {} ".format(i+1, tag1, tag2))

  except:
    print('Check if URL from {} is valid'.format(url))